In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3

In [15]:
for i  in d2.Link.values:
    index_names = d1[ d1['Link'] == i ].index
    d1.drop(index_names, inplace = True)

In [16]:
d1['Gender']='Women'
d2['Gender']='Men'

In [17]:
d=d1.append(d2)

In [18]:
d.isnull().sum()

Product             0
Link                0
Brand              19
Size Available      0
Selling Price       0
MRP                 0
Description         0
Primary_Image      10
Secondary_Image     4
Gender              0
dtype: int64

In [ ]:
data=pd.read_csv('All_SC.csv')

In [20]:
data.shape

(23216, 10)

In [21]:
data=data.dropna()

In [22]:
data=data.append(d)

In [24]:
data.shape

(23114, 10)

In [25]:
data.to_csv("All_Shopclues.csv",index=False)

In [26]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('All_ShopClues.db')
c = conn.cursor()

c.execute('CREATE TABLE products (Product,Link,Brand,Size Available,Selling Price,MRP,Gender,Description,Primary_Image,Secondary_Image)')
conn.commit()

data.to_sql('products', conn, if_exists='replace', index = False)
conn.close

C:\Users\alokt\anaconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


<function Connection.close>

In [7]:
new=pd.read_csv('All_Shopclues.csv')

In [8]:
new.shape

(23114, 10)

In [9]:
new.isnull().sum()

Product             0
Link                0
Brand              19
Size Available      0
Selling Price       0
MRP                 0
Gender              0
Description         0
Primary_Image      10
Secondary_Image     4
dtype: int64

In [10]:
new=new.dropna()

In [11]:
int(new['Selling Price'][0][1:])

247

In [12]:
len(new['Selling Price'])

23082

In [13]:
sell=[]
for i in range(len(new['Selling Price'])):
    sell.append(int(re.sub("\D",'',new['Selling Price'].values[i])))
    

In [14]:
mrp=[]
for i in range(len(new['MRP'])):
    mrp.append(int(re.sub("\D",'',new['MRP'].values[i])))

In [15]:
new['Selling Price']=sell
new['MRP']=mrp

In [16]:
new.head()

,Product,Link,Brand,Size Available,Selling Price,MRP,Gender,Description,Primary_Image,Secondary_Image
0,Sunshopping men's multicolor leatherite needle...,https://www.shopclues.com/sunshopping-mens-mul...,Sunshopping,"28 , 30 , 32 , 34 , 36 , 38",247,599,Men,Buy [product _name] Online at Best Prices from...,https://cdn.shopclues.com/images1/detailed/909...,https://cdn.shopclues.com/images1/detailed/909...
1,Sunshopping men's brown leatherite needle pin ...,https://www.shopclues.com/sunshopping-mens-bro...,Sunshopping,"28 , 30 , 32 , 36",171,499,Men,Buy [product _name] Online at Best Prices from...,https://cdn.shopclues.com/images1/detailed/890...,https://cdn.shopclues.com/images1/detailed/890...
2,UNU New Stylish Leather Strap 0060 Watch Gor B...,https://www.shopclues.com/unu-new-stylish-leat...,Unu,Not Available,152,999,Men,Buy UNU New Stylish Leather Strap 0060 Watch G...,https://cdn.shopclues.com/images1/detailed/968...,Not Available
3,Knoos Men Synthetic Leather Loafers,https://www.shopclues.com/knoos-men-synthetic-...,Knoos,9,427,1045,Men,Buy Knoos Men Synthetic Leather Loafers online...,https://cdn.shopclues.com/images/detailed/7901...,https://cdn.shopclues.com/images/detailed/7901...
4,Sunshopping Men's Brown Leatherite Auto Lock B...,https://www.shopclues.com/sunshopping-mens-bro...,Sunshopping,"28 , 30 , 32 , 34 , 36 , 38",266,599,Men,Buy [product _name] Online at Best Prices from...,https://cdn.shopclues.com/images1/detailed/971...,https://cdn.shopclues.com/images1/detailed/971...


In [17]:
data_1=pd.DataFrame({
    
    "Website":'https://www.shopclues.com/',
    "Product_Link":new['Link'].values,
    "Product_Name":new['Product'].values,
    "Product_Brand":new['Brand'].values,
    "Size_Avail":new['Size Available'].values,
    "Price":new['Selling Price'].values,
    "MRP":new['MRP'].values,
    "Gender":new['Gender'].values,
    "Description":new['Description'].values,
    "Primary_Image_Links":new['Primary_Image'].values,
    "Secondary_Image_Links":new['Secondary_Image'].values,
    "Affiliate_Link":''
    
})

In [18]:
data_1.to_csv('shopclues.csv',index=False)

In [80]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('ShopC.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')


data_1.to_sql('product_details', conn, if_exists='replace', index = False)    
conn.commit()
conn.close

<function Connection.close>

## Men Accessories

In [20]:
link=[]
products=['jewelry-watches-mens-jewellery','mens-accessories-socks','accessories-men-caps']
for i in products:
    c = 0
    while (c <400):
        c=c+1
        result=requests.get(f'https://www.shopclues.com/{i}.html?page={c}')
        g= result.content
        soup = BeautifulSoup(g,"html.parser")
        data=soup.find_all('div',class_='column col4')
        for j in data:
            try:
                link.append("https:"+j.find_all('a')[1].attrs['href'])
            except:
                pass

In [24]:
link=list(set(link))

In [63]:
len(link)

4536

In [62]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [64]:
error_link=[]
out_of_stock=[]
for i in link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
                
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'class':'o_price'})!=None):
                    MRP.append(s.find('span',{'class':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'class':"f_price"}).text)
            
            Price.append(s.find('span',{'class':'f_price'}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("ShopClues")
                    
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            Gender.append('Men')
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==0):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [65]:
error_link

['https://www.shopclues.com/crystal-pack-of-10-long-socks-for-men-139419480.html']

In [66]:
len(Secondary_Image_Link)

4525

In [67]:
data_1=pd.DataFrame({
    
    "Website":'https://www.shopclues.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [68]:
data_1.to_csv('Men_Acc.csv',index=False)

## Women Accessories

In [2]:
link=[]
products=['all-jewellery','womens-accessories']
for i in products:
    c = 0
    while (c <400):
        c=c+1
        result=requests.get(f'https://www.shopclues.com/{i}.html?page={c}')
        g= result.content
        soup = BeautifulSoup(g,"html.parser")
        data=soup.find_all('div',class_='column col4')
        for j in data:
            try:
                link.append("https:"+j.find_all('a')[1].attrs['href'])
            except:
                pass

In [3]:
link=list(set(link))
len(link)

2491

In [4]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [5]:
error_link=[]
out_of_stock=[]
for i in link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
                
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'class':'o_price'})!=None):
                    MRP.append(s.find('span',{'class':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'class':"f_price"}).text)
            
            Price.append(s.find('span',{'class':'f_price'}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("ShopClues")
                    
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            Gender.append('Women')
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==0):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [6]:
len(error_link)

16

In [7]:
data_2=pd.DataFrame({
    
    "Website":'https://www.shopclues.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [52]:
data_2.to_csv('Women_Acc.csv',index=False)

In [9]:
link=[]
products=['travel-and-luggage','case-and-cover-10']
for i in products:
    c = 0
    while (c <400):
        c=c+1
        result=requests.get(f'https://www.shopclues.com/{i}.html?page={c}')
        g= result.content
        soup = BeautifulSoup(g,"html.parser")
        data=soup.find_all('div',class_='column col4')
        for j in data:
            try:
                link.append("https:"+j.find_all('a')[1].attrs['href'])
            except:
                pass

In [10]:
link=list(set(link))

In [11]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [12]:
error_link=[]
out_of_stock=[]
for i in link:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        s=soup.find('div',class_='prd_mid_info')
        
        if(list(s.find('link',{'itemprop':'availability'}).attrs.values())[1]=='https://schema.org/InStock'):
            
            
                
            if(s.find('span',{'id':"sec_list_price_"})!=None):
                MRP.append(s.find('span',{'id':"sec_list_price_"}).text.split('MRP:')[1])
            else:
                if(s.find('span',{'class':'o_price'})!=None):
                    MRP.append(s.find('span',{'class':"o_price"}).text)
                else:
                    MRP.append(s.find('span',{'class':"f_price"}).text)
            
            Price.append(s.find('span',{'class':'f_price'}).text)
            Product_Link.append(i)
            Product_Name.append(s.find('h1',{'itemprop':'name'}).text.rstrip().lstrip())
            if(s.find('span',{'itemprop':'brand'}).text!=''):
                Product_Brand.append(s.find('span',{'itemprop':'brand'}).text)
            else:
                if(s.find('span',{'class':'pID'}).a!=None):
                    Product_Brand.append(s.find('span',{'class':'pID'}).a.text)
                else:
                    Product_Brand.append("ShopClues")
                    
            size=s.find('div',{'class':"f_variation"})
            if(size==None):
                Size_Available.append("Not Available")
            else:
                li=size.ul.find_all('span',{'class':'variant'})
                s_l=''
                for j in li:
                    s_l=s_l+j.text+" , "
                Size_Available.append(s_l[:-2])
            
            Description.append(s.find('span',{'itemprop':'description'}).text)
            Gender.append('Unisex')
            img=soup.find('div',class_='prd_img_gallery')
            img_list=img.find_all('li',class_="thumb_click")
            if(len(img_list)==0):
                Primary_Image_Link.append('Not Available')
                Secondary_Image_Link.append('Not Available')
            else:
                if(len(img_list)==1):
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append("Not Available")
                else:
                    Primary_Image_Link.append(img_list[0].a.attrs['data-zoom-image'])
                    Secondary_Image_Link.append(img_list[1].a.attrs['data-zoom-image'])
        else:
            out_of_stock.append(i)
            
    except:
        error_link.append(i)
        pass

In [13]:
data_3=pd.DataFrame({
    
    "Website":'https://www.shopclues.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [14]:
data_3.to_csv('Unisex_Acc.csv',index=False)

In [15]:
d1=pd.read_csv('Men_Acc.csv')

In [16]:
acc=d1.append(data_2)
acc=acc.append(data_3)

In [22]:
acc.shape

(9356, 12)

In [23]:
mrp=[]
for i in range(len(acc['MRP'])):
    mrp.append(int(re.sub("\D",'',acc['MRP'].values[i])))

In [25]:
sell=[]
for i in range(len(acc['Price'])):
    sell.append(int(re.sub("\D",'',acc['Price'].values[i])))
    

In [26]:
acc['Price']=sell
acc['MRP']=mrp

In [27]:
acc.shape

(9356, 12)

In [31]:
s=pd.read_csv('shopclues.csv')

In [32]:
s=s.append(acc)

In [33]:
s.shape

(32438, 12)

In [43]:
s.drop_duplicates(inplace=True)

In [44]:
s.shape

(32438, 12)

In [37]:
len(s['Product_Link'])

32438

In [58]:
s['Product_Brand']=s['Product_Brand'].replace('Not Available','ShopClues')

In [59]:
a=list(set(s['Product_Link']))

In [63]:
s.to_csv('All_Shop_Clues.csv',index=False)

In [62]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('All_Shop_Clues.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')


s.to_sql('product_details', conn, if_exists='replace', index = False)    
conn.commit()
conn.close

<function Connection.close>